In [1]:
# 0. Pacotes
import pandas as pd
from unicodedata import normalize
from bs4 import BeautifulSoup
import urllib.request
import networkx as nx
import matplotlib.pyplot as plt
import operator

In [3]:
# 1. Dados
df_reembolsos = pd.read_csv("Datasets/reembolso.csv", delimiter = ';')  # junção de tabelas baixadas em http://www2.camara.leg.br/transparencia/cota-para-exercicio-da-atividade-parlamentar/dados-abertos-cota-parlamentar
df_reembolsos = df_reembolsos[df_reembolsos.txtCNPJCPF.notnull()]  # desprezando casos em que o cnpj da empresa é nulo.
df_reembolsos = df_reembolsos[df_reembolsos["numAno"] > 2015]  # focando nos reembolsos a partir de 2012

/home/gustavo/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_receitas = pd.read_csv("Datasets/despesas_candidatos_2014_brasil.csv", delimiter=';')  # csv baixado em http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais
df_receitas = df_receitas[df_receitas["Cargo"] == "Deputado Federal"] # somente deputados federais

/home/gustavo/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# 1.1. Criando dicionário com identificador e nome completo do deputado (será necessário porque a tabela de reembolsos não tem o nome completo, só o apelido)
page = urllib.request.urlopen("http://www.camara.leg.br/SitCamaraWS/Deputados.asmx/ObterDeputados")  # link com os dados básicos dos deputados
soup = BeautifulSoup(page)
lista_dados_deputados = soup.find_all("deputado")
def remover_acentos(txt):
    return normalize("NFKD", txt).encode("ASCII", "ignore").decode("ASCII")

deputados_dict = {}
for i in range(0,len(lista_dados_deputados)):
    deputados_dict[lista_dados_deputados[i].idecadastro.string] = [remover_acentos(lista_dados_deputados[i].nome.string), lista_dados_deputados[i].urlfoto.string]

/home/gustavo/Projetos_Python/Anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/gustavo/Projetos_Python/Anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [6]:
df_reembolsos

,txNomeParlamentar,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,...,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,"ideDocumento,,,"
403585,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,1.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0.0,...,7.0,2017.0,0.0,NaN,NaN,1401321.0,NaN,NaN,2812.0,6328475
403586,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,1.0,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0.0,...,10.0,2017.0,0.0,NaN,NaN,1436042.0,NaN,NaN,2812.0,6423547
403587,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,9.0,PASSAGENS AÉREAS,0.0,...,7.0,2017.0,0.0,MÁRCIA OLIVEIRA GOMES;,SÃO PAULO / BRASÍLIA,1401049.0,NaN,NaN,2812.0,6327499
403588,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,9.0,PASSAGENS AÉREAS,0.0,...,7.0,2017.0,0.0,MÁRCIA OLIVEIRA GOMES;,BRASÍLIA/SÃO PAULO,1401049.0,NaN,NaN,2812.0,6327494
403589,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,13.0,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0.0,...,8.0,2017.0,0.0,NaN,NaN,1411388.0,NaN,NaN,2812.0,6356388
403590,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,13.0,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0.0,...,10.0,2017.0,0.0,NaN,NaN,1431815.0,NaN,NaN,2812.0,6412955
403591,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,13.0,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0.0,...,5.0,2017.0,0.0,NaN,NaN,1382939.0,NaN,NaN,2812.0,6276476
403592,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,13.0,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0.0,...,11.0,2017.0,0.0,NaN,NaN,1445004.0,NaN,NaN,2812.0,6447018
403593,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,13.0,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0.0,...,7.0,2017.0,0.0,NaN,NaN,1405052.0,NaN,NaN,2812.0,6336332
403594,LID.GOV-CD,NaN,NaN,2015.0,NaN,NaN,55.0,13.0,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,0.0,...,10.0,2017.0,0.0,NaN,NaN,1435313.0,NaN,NaN,2812.0,6421575


In [8]:
# 1.2. Tratamento dos dados
# 1.2.1. Ajustando nomes dos campos
df_reembolsos.columns = ['nome_parlamentar', 'num_agente_politico', 'carteira_parlamentar', 'mandato', 'uf',
                        'partido', 'legislatura', 'sub_cota', 'descricao', 'especificacao_subcota',
                        'descricao_especificacao', 'fornecedor', 'cnpj_cpf_empresa', 'numero_documento_reebolso', 
                         'tipo_documento', 'data_emissao', 'valor_documento', 'valor_glosa', 'valor_liquido',
                        'mes', 'ano', 'parcelado', 'passageiro', 'trecho', 'lote', 'num_ressarcimento', 
                        'vlr_restituicao', 'deputado_id', 'documento_id']

In [9]:
df_receitas.columns = ['cod_eleicao', 'desc_eleicao', 'data_hora', 'CNPJ_prestador_conta', 'seq_candidato', 'uf', 'partido',
                      'numero_candidato', 'cargo', 'nome_candidato', 'cpf_candidato', 'tipo_documento', 
                      'num_documento', 'cpf_cnpj_fornecedor', 'nome_fornecedor', 'nome_fornecedor_rf',
                      'cod_setor_economico_fornencedor', 'setor_economico_fornecedor', 'data_despesa', 
                      'valor_despesa', 'tipo_despesa', 'descricao_despesa', '']

In [10]:
# 1.2.2. Criando tabelas agregadas
df_reembolso_resumo = df_reembolsos.groupby(['partido', 'uf', 'deputado_id', 'fornecedor', 'cnpj_cpf_empresa'
                                            ]).agg({'valor_liquido':['sum','count']}).reset_index()

In [11]:
df_receitas_resumo = df_receitas.groupby(['partido', 'uf', 'nome_candidato', 'nome_fornecedor', 
                                          'cpf_cnpj_fornecedor']).agg({'valor_despesa':['sum', 'count']}).reset_index()

In [12]:
df_reembolso_resumo

partido  uf deputado_id  \
                                        
0              AVANTE  AL      1332.0   
1              AVANTE  AL      1332.0   
2              AVANTE  AL      1332.0   
3              AVANTE  AL      1332.0   
4              AVANTE  AL      1332.0   
5              AVANTE  AL      1332.0   
6              AVANTE  AL      1332.0   
7              AVANTE  AL      1332.0   
8              AVANTE  AL      1332.0   
9              AVANTE  AL      1332.0   
10             AVANTE  AL      1332.0   
11             AVANTE  AL      1332.0   
12             AVANTE  AL      1332.0   
13             AVANTE  AL      1332.0   
14             AVANTE  AL      1332.0   
15             AVANTE  AL      1332.0   
16             AVANTE  AL      1332.0   
17             AVANTE  AL      1332.0   
18             AVANTE  AL      1332.0   
19             AVANTE  AL      1332.0   
20             AVANTE  AL      1332.0   
21             AVANTE  AL      1332.0   
22             AVANTE  AL      1332.0   
23             AVANTE  AL      1332.0   
24             AVANTE  AL      1332.0   
25             AVANTE  AL      1332.0   
26             AVANTE  AL      1332.0   
27             AVANTE  AL      1332.0   
28             AVANTE  AL      1332.0   
29             AVANTE  AL      1332.0   
...               ...  ..         ...   
106907  SOLIDARIEDADE  SP      1903.0   
106908  SOLIDARIEDADE  SP      1903.0   
106909  SOLIDARIEDADE  SP      1903.0   
106910  SOLIDARIEDADE  SP      1903.0   
106911  SOLIDARIEDADE  SP      1903.0   
106912  SOLIDARIEDADE  SP      1903.0   
106913  SOLIDARIEDADE  SP      1903.0   
106914  SOLIDARIEDADE  SP      1903.0   
106915  SOLIDARIEDADE  SP      1903.0   
106916  SOLIDARIEDADE  SP      1903.0   
106917  SOLIDARIEDADE  SP      1903.0   
106918  SOLIDARIEDADE  SP      1903.0   
106919  SOLIDARIEDADE  SP      1903.0   
106920  SOLIDARIEDADE  SP      1903.0   
106921  SOLIDARIEDADE  SP      1903.0   
106922  SOLIDARIEDADE  SP      1903.0   
106923  SOLIDARIEDADE  SP      1903.0   
106924  SOLIDARIEDADE  SP      1903.0   
106925  SOLIDARIEDADE  SP      1903.0   
106926  SOLIDARIEDADE  SP      1903.0   
106927  SOLIDARIEDADE  SP      1903.0   
106928  SOLIDARIEDADE  SP      1903.0   
106929  SOLIDARIEDADE  SP      1903.0   
106930  SOLIDARIEDADE  SP      1903.0   
106931  SOLIDARIEDADE  SP      1903.0   
106932  SOLIDARIEDADE  SP      1903.0   
106933  SOLIDARIEDADE  SP      1903.0   
106934  SOLIDARIEDADE  SP      1903.0   
106935  SOLIDARIEDADE  SP      1903.0   
106936  SOLIDARIEDADE  SP      1903.0   

                                               fornecedor    cnpj_cpf_empresa  \
                                                                                
0                              A.B.LESSA NETO & CIA LTDA.  055.625.890/0013-5   
1                      AAMM COMERCIO DE COMBUSTIVEIS LTDA  091.649.510/0017-0   
2                                      ADILZA S. F. RAMOS  013.329.220/0016-9   
3                               ALEX SANDRO DOS SANTOS ME  107.826.560/0016-7   
4                                ALMIR E SILVA LTDA - EPP  049.791.690/0019-6   
5                                  ALMIR E SILVA LTDA EPP  049.791.690/0019-6   
6                            AUTO POSTO ALAZAO LTDA - EPP  190.343.670/0015-4   
7                             AUTO POSTO FOLHA MIUDA LTDA  134.784.810/0019-7   
8                             AUTO POSTO M M GARROTE LTDA  087.389.940/0015-0   
9                           AUTO POSTO PAGUE MENOS - LTDA  157.392.140/0015-1   
10                      AUTOPEL AUTO POSTO PROGRESSO LTDA  084.221.150/0018-6   
11                                 Auto Posto Lagoa - EPP  245.868.030/0013-8   
12                                  BLOG FERREIRA DELMIRO  238.017.050/0010-3   
13          COMERCIAL DE COMB. E LUBRIFICANTES VITAL LTDA  028.176.550/0018-2   
14      COOPERATIVA DE PRODUCAO E TRABALHO DOS JORNALI...  089.510.560/0013-3   
15      CORREIOS - CAIXA DE ENCOMENDAS BÁSICA/CONVENC